In [2]:
import pandas as pd 
from datetime import datetime, date, time, timedelta

df = pd.read_csv("crime.csv") 

In [3]:
df['DATE'], df['TIME'] = df['FIRST_OCCURRENCE_DATE'].str.split(' ', 1).str
df['DATE'] = pd.to_datetime(df['DATE'])
df['day_of_week'] = df['DATE'].dt.day_name()

In [4]:
df_property = df[['OFFENSE_CATEGORY_ID', 'TIME', 'day_of_week','NEIGHBORHOOD_ID','GEO_LON','GEO_LAT']]
df_property = df_property.loc[(df_property.OFFENSE_CATEGORY_ID == 'arson')|(df_property.OFFENSE_CATEGORY_ID == 'theft-from-motor-vehicle')|(df_property.OFFENSE_CATEGORY_ID == 'larceny')|(df_property.OFFENSE_CATEGORY_ID == 'burglary')|(df_property.OFFENSE_CATEGORY_ID == 'auto-theft')].copy()
df_property_am =  df_property.loc[df_property.TIME.str[-2:] == "AM" ].copy()
df_property_am['TIME'] = [x[:-3] for x in df_property_am['TIME']]
df_property_am['TIME'] =  pd.to_datetime(df_property_am['TIME'], format='%H:%M:%S')

df_property = df_property[['OFFENSE_CATEGORY_ID', 'TIME', 'day_of_week','NEIGHBORHOOD_ID','GEO_LON','GEO_LAT']]
df_property = df_property.loc[(df_property.OFFENSE_CATEGORY_ID == 'arson')|(df_property.OFFENSE_CATEGORY_ID == 'theft-from-motor-vehicle')|(df_property.OFFENSE_CATEGORY_ID == 'larceny')|(df_property.OFFENSE_CATEGORY_ID == 'burglary')|(df_property.OFFENSE_CATEGORY_ID == 'auto-theft')].copy()
df_property_pm =  df_property.loc[df_property.TIME.str[-2:] == "PM" ].copy()
df_property_pm['TIME'] = [x[:-3] for x in df_property_pm['TIME']]
df_property_pm['TIME'] =  pd.to_datetime(df_property_pm['TIME'], format='%H:%M:%S')
df_property_pm['TIME'] = df_property_pm['TIME'] + timedelta(hours=12)


In [5]:
final_property = pd.concat([df_property_am, df_property_pm], ignore_index=True, sort =False)

block1 = '09/19/18 04:00:00'
block2 = '09/19/18 08:00:00'
block3 = '09/19/18 12:00:00'
block4 = '09/19/18 16:00:00'
block5 = '09/19/18 20:00:00'
block6 = '09/19/18 23:59:00'
datetime_object1 = datetime.strptime(block1, '%m/%d/%y %H:%M:%S')
datetime_object2 = datetime.strptime(block2, '%m/%d/%y %H:%M:%S')
datetime_object3 = datetime.strptime(block3, '%m/%d/%y %H:%M:%S')
datetime_object4 = datetime.strptime(block4, '%m/%d/%y %H:%M:%S')
datetime_object5 = datetime.strptime(block5, '%m/%d/%y %H:%M:%S')
datetime_object6 = datetime.strptime(block6, '%m/%d/%y %H:%M:%S')


In [6]:
final_property.loc[final_property['TIME'].dt.time <= datetime_object1.time(), 'TIME'] = datetime_object1

final_property.loc[(final_property['TIME'].dt.time > datetime_object1.time())&(final_property['TIME'].dt.time <= datetime_object2.time()), 'TIME'] = datetime_object2

final_property.loc[(final_property['TIME'].dt.time > datetime_object2.time())&(final_property['TIME'].dt.time <= datetime_object3.time()), 'TIME'] = datetime_object3

final_property.loc[(final_property['TIME'].dt.time > datetime_object3.time())&(final_property['TIME'].dt.time <= datetime_object4.time()), 'TIME'] = datetime_object4

final_property.loc[(final_property['TIME'].dt.time > datetime_object4.time())&(final_property['TIME'].dt.time <= datetime_object5.time()), 'TIME'] = datetime_object5

final_property.loc[(final_property['TIME'].dt.time > datetime_object5.time())&(final_property['TIME'].dt.time <= datetime_object6.time()), 'TIME'] = datetime_object6



final_property['TIME'] = [d.time() for d in final_property['TIME']]
final_property

,OFFENSE_CATEGORY_ID,TIME,day_of_week,NEIGHBORHOOD_ID,GEO_LON,GEO_LAT
0,theft-from-motor-vehicle,16:00:00,Wednesday,highland,-105.015451,39.757627
1,theft-from-motor-vehicle,08:00:00,Sunday,ruby-hill,-104.997720,39.694578
2,theft-from-motor-vehicle,04:00:00,Sunday,highland,-105.014275,39.764565
3,auto-theft,12:00:00,Saturday,west-colfax,-105.032978,39.737471
4,auto-theft,08:00:00,Monday,dia,-104.700816,39.831895
...,...,...,...,...,...,...
148113,theft-from-motor-vehicle,20:00:00,Tuesday,cole,-104.970490,39.770122
148114,larceny,20:00:00,Sunday,north-capitol-hill,-104.981335,39.746330
148115,auto-theft,20:00:00,Sunday,sunnyside,-105.011567,39.775002
148116,theft-from-motor-vehicle,20:00:00,Friday,speer,-104.974448,39.716818


In [7]:
property_csv = final_property[['day_of_week','NEIGHBORHOOD_ID', 'TIME']]
property_plot = final_property[['day_of_week','NEIGHBORHOOD_ID', 'TIME','GEO_LON','GEO_LAT']]
property_csv = property_csv.sample(40000)
property_csv.to_csv('Property.csv')
property_plot.to_csv('Property_Plot.csv')


!python apriori.py -f 'Property.csv' -s 0.002

item: ('Wednesday', '16:00:00', 'stapleton') , 0.002
item: ('Saturday', '16:00:00', 'stapleton') , 0.002
item: ('Friday', 'five-points', '20:00:00') , 0.002
item: ('stapleton', 'Monday', '20:00:00') , 0.002
item: ('20:00:00', 'Saturday', 'stapleton') , 0.002
item: ('20:00:00', 'Thursday', 'stapleton') , 0.002
item: ('20:00:00', 'Wednesday', 'stapleton') , 0.003
item: ('20:00:00', 'Friday', 'stapleton') , 0.003
